In [2]:
import pycaret
pycaret.__version__

'3.2.0'

In [1]:
import lightgbm
lightgbm.__version__

'4.3.0'

## 데이터 불러오기 

In [3]:
import pandas as pd

titanic_df = pd.read_csv('./data/titanic_train.csv')
titanic_df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [5]:
# 객체의 타입 확인
print(type(titanic_df))

<class 'pandas.core.frame.DataFrame'>


In [6]:
# shape : 행과 열을 튜플 형태로 반환
titanic_df.shape

(891, 12)

In [7]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [8]:
# describe : 칼럼별 숫자형 데이터값의 n-percentile 분포도, 평균값, 최댓값, 최솟값을 나타냄
titanic_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [9]:
titanic_df['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [10]:
type(titanic_df['Pclass'].value_counts())

pandas.core.series.Series

In [15]:
temp = titanic_df['Pclass'].value_counts()
temp

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [16]:
temp.index

Int64Index([3, 1, 2], dtype='int64')

In [17]:
temp.values

array([491, 216, 184])

In [18]:
titanic_df['Cabin'].value_counts()

B96 B98        4
G6             4
C23 C25 C27    4
C22 C26        3
F33            3
              ..
E34            1
C7             1
C54            1
E36            1
C148           1
Name: Cabin, Length: 147, dtype: int64

In [22]:
# 결측치 포함
titanic_df['Cabin'].value_counts(dropna=False)

NaN            687
C23 C25 C27      4
G6               4
B96 B98          4
C22 C26          3
              ... 
E34              1
C7               1
C54              1
E36              1
C148             1
Name: Cabin, Length: 148, dtype: int64

In [21]:
# 결측치 포함 x
titanic_df['Cabin'].value_counts(dropna=True)

B96 B98        4
G6             4
C23 C25 C27    4
C22 C26        3
F33            3
              ..
E34            1
C7             1
C54            1
E36            1
C148           1
Name: Cabin, Length: 147, dtype: int64

## 교재 50~53p
- numpy, list, dictionary와 DataFrame의 상호변환이 가능

In [25]:
# to_numpy() ==> 굳이 필요없음
titanic_df.values # numpy로 변환
# titanic_df.values.tolist() # list로 변환
# temp.values.tolist()

# 데이터프레임을 딕셔너리로 변환
# titanic_df.to_dict('list')

array([[1, 0, 3, ..., 7.25, nan, 'S'],
       [2, 1, 1, ..., 71.2833, 'C85', 'C'],
       [3, 1, 3, ..., 7.925, nan, 'S'],
       ...,
       [889, 0, 3, ..., 23.45, nan, 'S'],
       [890, 1, 1, ..., 30.0, 'C148', 'C'],
       [891, 0, 3, ..., 7.75, nan, 'Q']], dtype=object)

# DataFrame의 칼럼 데이터 세트 생성과 수정

In [28]:
# data['새로운컬럼명'] = 연산

## DataFrame 삭제
- axis = 1 ==> 지정된 컬럼 삭제
- axis = 0 ==> 지정된 index(=행) 삭제
- 57p, inplace=True, 객체를 저장하면서 업데이트

## set_option()
- streamlit을 활용하여 테이블을 깔끔하게 보여주는 경우에는 도움이 되겠으나 분석을 할 때는 크게 활용가치가 없음

## 데이터 셀렉션 및 필터링
- .loc 문법만 잘 기억해도 큰 문제가 없음
- keyError ==> 해당 데이터 프레임의 컬럼명이 없는 상태

## 결손 데이터 처리하기
- Missing Data

In [29]:
titanic_df.isna().head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,True,False


In [30]:
titanic_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### fillna()로 결손 데이터(NULL값) 대체하기 

In [31]:
titanic_df['Cabin'] = titanic_df['Cabin'].fillna('C000')
titanic_df['Age'] = titanic_df['Age'].fillna(titanic_df['Age'].mean())
titanic_df['Embarked'] = titanic_df['Embarked'].fillna('S')

In [32]:
titanic_df.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [34]:
# value_counts()는 자동으로 내림차순 정렬을 해줌
titanic_df['Embarked'].value_counts().index[0]

'S'

In [35]:
# titanic_df['Embarked'] = titanic_df['Embarked'].fillna('S')
# 가장 많이 나온 값으로 대체하기
titanic_df['Embarked'] = titanic_df['Embarked'].fillna(titanic_df['Embarked'].value_counts().index[0])

In [37]:
# 빈도수가 가장 높은 것을 뽑음
titanic_df['Embarked'].mode()
titanic_df['Embarked'].mode()[0]

'S'

In [38]:
# 가장 많이 나온 값으로 대체하기
titanic_df['Embarked'] = titanic_df['Embarked'].fillna(titanic_df['Embarked'].mode()[0])

## apply lambda 식으로 데이터 가공
- 1회성 사용자 정의 함수

In [39]:
def get_square(a):
    return a**2

get_square(3)

9

In [40]:
lambda_square = lambda x : x ** 2
lambda_square(6)

36

In [42]:
a = [1, 2, 3, 4, 5, 6]
squares = map(lambda x : x ** 2, a)
list(squares)

[1, 4, 9, 16, 25, 36]

In [44]:
titanic_df['Name_len'] = titanic_df['Name'].apply(lambda x : len(x))
titanic_df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_len
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,C000,S,23
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,51
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,C000,S,22


## apply()
- 행 단위로 연산을 할 때, apply() 사용
  + lambda : 간단한 연산을 할 때
  + UDF(User Defined Function) : if 조건문이 여러번 사용될 것 같은 상황

In [45]:
def get_category(age):
    cat = ''
    if age <= 5: cat = 'Baby'
    elif age <= 12: cat = 'Child'
    elif age <= 18: cat = 'Teenager'
    elif age <= 25: cat = 'Student'
    elif age <= 35: cat = 'Young Adult'
    elif age <= 60: cat = 'Adult'
    else : cat = 'Elderly'
    
    return cat

In [49]:
titanic_df['Age_category'] = titanic_df['Age'].apply(lambda x : get_category(x))
titanic_df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_len,Age_category
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,C000,S,23,Student
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,51,Adult
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,C000,S,22,Young Adult


## 사이킷런 (Scikit-Learn)
- Python 머신러닝 라이브러리
- 엑셀 데이터 ==> 정형 데이터 ==> 머신러닝 디폴트 라이브러리
  + 텐서플로, 케라스, 파이토치 딥러닝 전문 라이브러리 (비정형 데이터)
  + 이미지 (비디오 영상), 자연어 (텍스트 데이터, LLM)
- 파이썬 기반의 다른 머신러닝 패키지도 사이킷런 스타일의 API를 지향한다
  + XGBoost, LightGBM, CatBoost 이런 종류의 알고리즘은 사이킷런 내부 라이브러리는 아님.
  + XGBoost, LightGBM, CatBoost 알고리즘도 마치 사이킷런 클래스처럼 사용이 가능하도록 지원
 
- 가장 큰 단점
  + GPU 연산을 못함 / CPU 연산만 수행

In [50]:
import sklearn

sklearn.__version__

'1.2.2'

### 붓꽃 품종 예측하기
- 3가지 중 한개를 예측

In [62]:
from sklearn.datasets import load_iris 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 

In [63]:
import pandas as pd # 데이터 프레임 생성 목적

iris = load_iris() # numpy 배열로 데이터가 존재함

# 독립변수 = 설명변수
iris_data = iris.data # 딕셔너리 key값만 출력

# 지도학습 = 종속변수
iris_label = iris.target # 0은 setosa, 1은 versicolor, 2는 virginica 품종
iris_label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [66]:
iris_df = pd.DataFrame(data=iris_data, columns = iris.feature_names) # 독립변수의 컬럼명
iris_df['label'] = iris.target
iris_df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


## 생각해야 하는 것
- 종속변수가 문자로 되어 있다.
  + 수단과 방법을 가리지 않고, 종속변수를 숫자로 변형을 해야함 !

## 훈련데이터와 테스트 데이터로 분리
- 정석 : 훈련(train) / 검증(validation) / 테스트(test)
- 야매 : 훈련 / 테스트

In [68]:
# X_train : 훈련 데이터의 독립변수(설명변수)
# X_test : 테스트 데이터의 독립변수(설명변수)
# y_train : 훈련 데이터의 종속변수
# y_test : 테스트 데이터의 종속변수

# iris_data : 전체 데이터 독립변수
# iris_label : 전체 데이터 종속변수
X_train, X_test, y_train, y_test = train_test_split(iris_data, 
                                                    iris_label, 
                                                    test_size = 0.2, 
                                                    random_state = 11)

## 모델 학습
- DecisionTreeClassfier 객체 생성

In [71]:
dt_clf = DecisionTreeClassifier(random_state=11)

# 학습 수행
dt_clf.fit(X_train, y_train) # 훈련 데이터의 독립변수, 종속변수

DecisionTreeClassifier(random_state=11)

## 모델 학습 후 예측
- 테스트 데이터 셋으로 예측 수행

In [72]:
# p.90
pred = dt_clf.predict(X_test) # 테스트 데이터의 독립변수만을 가지고 예측
pred

array([2, 2, 1, 1, 2, 0, 1, 0, 0, 1, 1, 1, 1, 2, 2, 0, 2, 1, 2, 2, 1, 0,
       0, 1, 0, 0, 2, 1, 0, 1])

In [73]:
# X_test와 일치하는지 확인
y_test

array([2, 2, 2, 1, 2, 0, 1, 0, 0, 1, 2, 1, 1, 2, 2, 0, 2, 1, 2, 2, 1, 0,
       0, 1, 0, 0, 2, 1, 0, 1])

In [74]:
# X_test와 y_test의 정확도 확인
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.9333333333333333

## 사이킷런 코드 형식
- scklearn.pipeline 방식대로 작성하는 것을 매우 강력하게 추천
  + 공식문서에서 매우 권장하는 사항
  + 빅데이터 분석기사 : 머신러닝 관련된 시험 코드 암기 (최소 30점 / 40점 만점)
 
- 수많은 머신러닝 교재에서 pipeline 잘 소개하는 책은 거의 없음
  + 빅데이터 분석기사 교재 (X)